In [1]:
import json
import glob
import os

In [26]:
config_filename = "multi_resample_config_v1.json"

config = dict()

model_config = dict()
model_config["name"] = "DynUNet"  # network model name from MONAI
# set the network hyper-parameters
model_config["in_channels"] = 4  # 4 input images for the BraTS challenge
model_config["out_channels"] = 1   # whole tumor, tumor core, enhancing tumor
model_config["spatial_dims"] = 3   # 3D input images
model_config["deep_supervision"] = False  # do not check outputs of lower layers
model_config["strides"] = [[1, 1, 1], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2]][:-1]  # number of downsampling convolutions
model_config["filters"] = [64, 96, 128, 192, 256, 384, 512, 768, 1024][:len(model_config["strides"])]  # number of filters per layer
model_config["kernel_size"] = [[3, 3, 3]] * len(model_config["strides"])  # size of the convolution kernels per layer
model_config["upsample_kernel_size"] = model_config["strides"][1:]  # should be the same as the strides

# put the model config in the main config
config["model"] = model_config

config["optimizer"] = {'name': 'Adam', 
                       'lr': 0.001}  # initial learning rate

# define the loss
config["loss"] = {'name': 'DiceLoss', # from Monai
                  'include_background': True,  # we do not have a label for the background, so this should be true (by "include background" monai means include channel 0)
                  'sigmoid': True,  # transform the model logits to activations
                  'batch': False}  

# set the cross validation parameters
config["cross_validation"] = {'folds': 5,  # number of cross validation folds
                              'seed': 25},  # seed to make the generation of cross validation folds consistent across different trials
# set the scheduler parameters
config["scheduler"] = {'name': 'ReduceLROnPlateau', 
                       'patience': 20,  # wait 10 epochs with no improvement before reducing the learning rate
                       'factor': 0.5,   # multiply the learning rate by 0.5
                       'min_lr': 1e-08}  # stop reducing the learning rate once it gets to 1e-8

# set the dataset parameters
config["dataset"] = {'name': 'SegmentationDatasetPersistent',  # 'Persistent' means that it will save the preprocessed outputs generated during the first epoch
# However, using 'Persistent', does also increase the time of the first epoch compared to the other epochs, which should run faster
  'desired_shape': [192, 192, 192],  # resize the images to this shape, increase this to get higher resolution images (increases computation time and memory usage)
  'labels': [1],  # 1: tumor
  'orientation': 'RAS',  # Force all the images to be the same orientation (Right-Anterior-Suppine)
  'normalization': 'NormalizeIntensityD',  # z score normalize the input images to zero mean unit standard deviation
  'normalization_kwargs': {'channel_wise': True, "nonzero": False},  # perform the normalization channel wise and include the background
  'resample': True,  # resample the images when resizing them, otherwise the resize could crop out regions of interest
  'crop_foreground': True,  # crop the foreground of the images
  'foreground_percentile': 0.9,  # aggressive foreground cropping to make sure the empty space is taken out of the images
  'training':  # the following arguments will only be applied to the training data.
    {
    'spatial_augmentations': [{'name': 'RandFlipD', 'spatial_axis': 0, 'prob': 0.5},
                              {'name': 'RandFlipD', 'spatial_axis': 1, 'prob': 0.5},
                              {'name': 'RandRotateD', 'prob': 0.5, 'range_x': 0.2, 'range_y': 0.2, 'range_z': 0.2}],
    'intensity_augmentations': [{'name': 'RandScaleIntensityD', 'factors': 0.1, 'prob': 1.0},
                                {'name': 'RandShiftIntensityD', 'offsets': 0.1, 'prob': 1.0}],
    }
                    }
config["training"] = {'batch_size': 2,  # number of image/label pairs to read at a time during training
  'validation_batch_size': 2,  # number of image/label pairs to read at atime during validation
  'amp': False,  # don't set this to true unless the model you are using is setup to use automatic mixed precision (AMP)
  'early_stopping_patience': None,  # stop the model early if the validaiton loss stops improving
  'n_epochs': 1000,  # number of training epochs, reduce this if you don't want training to run as long
  'save_every_n_epochs': None,  # save the model every n epochs (otherwise only the latest model will be saved)
  'save_last_n_models': None,  # save the last n models 
  'save_best': True}  # save the model that has the best validation loss

In [92]:
# get the training filenames
config["training_filenames"] = list()
ground_truth_filenames = sorted(glob.glob("./aligned/*/*/*NB*.nii*"))
for label_filename in ground_truth_filenames:
    subject, visit = label_filename.split("/")[-3:-1]
    filenames = [os.path.abspath(fn) for fn in sorted(glob.glob(os.path.join(os.path.dirname(label_filename), "*")))]
    n_features = len(filenames) 
    feature_modalities = ["_".join(fn.split("_")[3:-1]).strip("8 ").lower() for fn in filenames]
    t1_filename = filenames[feature_modalities.index("T1_gd".lower())]
    assert os.path.exists(t1_filename)
    
    if len(feature_modalities) < 5:
        continue
    
    if "T2".lower() not in feature_modalities:
        for filename in filenames:
            if "T2" in filename:
                t2_fn = filename
        # print(t2_fn)
        print(feature_modalities)
    else:
        t2_fn = filenames[feature_modalities.index("T2".lower())]
    assert os.path.exists(t2_fn)
    
    if "DWI_b0".lower() not in feature_modalities:
        for filename in filenames:
            if "DWI" in filename and "b0" in filename:
                dwi_b0 = filename
        # print(dwi_b0)
    else:
        dwi_b0 = filenames[feature_modalities.index("DWI_b0".lower())]
    assert os.path.exists(dwi_b0)

    if "DWI_b100".lower() not in feature_modalities:
        for filename in filenames:
            if "DWI" in filename and "b100" in filename:
                dwi_b100 = filename
        # print(dwi_b100)
    else:
        dwi_b100 = filenames[feature_modalities.index("DWI_b100".lower())]
    assert os.path.exists(dwi_b100)

    # print(t2_fn)
    config["training_filenames"].append({"image": [t1_filename, t2_fn, dwi_b0, dwi_b100], "label": label_filename})
with open(config_filename, "w") as op:
    json.dump(config, op, indent=4)

['dwi_b0', 'dwi_b100', 'nb', 't1_gd', '']


In [68]:
from monai.transforms import LoadImage
import tqdm
loader = LoadImage(image_only=True)
for item in tqdm.tqdm(config["training_filenames"][62:]):
    loader(item["image"])

 20%|██        | 2/10 [00:08<00:33,  4.18s/it]


RuntimeError: affine matrix of all images should be the same for channel-wise concatenation. Got [[-9.09744143e-01  4.17321585e-02  6.15315586e-02  1.63692520e+02]
 [-3.92322913e-02 -9.08636808e-01  9.93195400e-02  1.44127029e+02]
 [ 3.97835933e-02  5.82573116e-02  1.50500500e+00 -5.72437683e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]] and [[-9.11458313e-01 -4.17198098e-05  1.49980042e-04  1.73884186e+02]
 [ 4.22855992e-05 -9.11440492e-01  9.44507215e-03  1.53154114e+02]
 [ 9.02942920e-05  5.70288487e-03  1.50952148e+00 -4.98229950e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]].

In [69]:
loader = LoadImage(image_only=True)
for fn in item['image']:
    image = loader(fn)
    print(image.affine)

tensor([[-9.1146e-01, -4.1720e-05,  1.4998e-04,  1.7388e+02],
        [ 4.2286e-05, -9.1144e-01,  9.4451e-03,  1.5315e+02],
        [ 9.0294e-05,  5.7029e-03,  1.5095e+00, -4.9823e+02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],
       dtype=torch.float64)
tensor([[-9.0974e-01,  4.1732e-02,  6.1532e-02,  1.6369e+02],
        [-3.9232e-02, -9.0864e-01,  9.9320e-02,  1.4413e+02],
        [ 3.9784e-02,  5.8257e-02,  1.5050e+00, -5.7244e+02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],
       dtype=torch.float64)
tensor([[-9.1146e-01, -4.1720e-05,  1.4998e-04,  1.7388e+02],
        [ 4.2286e-05, -9.1144e-01,  9.4451e-03,  1.5315e+02],
        [ 9.0294e-05,  5.7029e-03,  1.5095e+00, -4.9823e+02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],
       dtype=torch.float64)
tensor([[-9.1146e-01, -4.1720e-05,  1.4998e-04,  1.7388e+02],
        [ 4.2286e-05, -9.1144e-01,  9.4451e-03,  1.5315e+02],
        [ 9.0294e-05,  5.7029e-03,  1.5095e+0

In [76]:
glob.glob("/lustre/work/aizenberg/dgellis/MICCAI23/code/3DUnetCNN/examples/sppin/aligned/PT_81/*/*")

['/lustre/work/aizenberg/dgellis/MICCAI23/code/3DUnetCNN/examples/sppin/aligned/PT_81/20190424/PT_81_DWI_b100_20190424.nii.gz',
 '/lustre/work/aizenberg/dgellis/MICCAI23/code/3DUnetCNN/examples/sppin/aligned/PT_81/20190424/PT_81_T2_20190424.nii.gz',
 '/lustre/work/aizenberg/dgellis/MICCAI23/code/3DUnetCNN/examples/sppin/aligned/PT_81/20190424/PT_81_DWI_b0_20190424.nii.gz',
 '/lustre/work/aizenberg/dgellis/MICCAI23/code/3DUnetCNN/examples/sppin/aligned/PT_81/20190424/PT_81_NB_20190424.nii.gz',
 '/lustre/work/aizenberg/dgellis/MICCAI23/code/3DUnetCNN/examples/sppin/aligned/PT_81/20190424/PT_81_T1_gd_20190424.nii',
 '/lustre/work/aizenberg/dgellis/MICCAI23/code/3DUnetCNN/examples/sppin/aligned/PT_81/20200129/PT_81_NB_20200129.nii.gz',
 '/lustre/work/aizenberg/dgellis/MICCAI23/code/3DUnetCNN/examples/sppin/aligned/PT_81/20200129/PT_81_DWI_b0_20200129.nii.gz',
 '/lustre/work/aizenberg/dgellis/MICCAI23/code/3DUnetCNN/examples/sppin/aligned/PT_81/20200129/PT_81_T1_gd_20200129.nii',
 '/lustre/

In [74]:
config['training_filenames'][63:66]

[{'image': ['/lustre/work/aizenberg/dgellis/MICCAI23/code/3DUnetCNN/examples/sppin/aligned/PT_81/20200129/PT_81_T1_gd_20200129.nii',
   '/lustre/work/aizenberg/dgellis/MICCAI23/code/3DUnetCNN/examples/sppin/aligned/PT_81/20200129/PT_81_T2_20200129.nii.gz',
   '/lustre/work/aizenberg/dgellis/MICCAI23/code/3DUnetCNN/examples/sppin/aligned/PT_81/20200129/PT_81_DWI_b0_20200129.nii.gz',
   '/lustre/work/aizenberg/dgellis/MICCAI23/code/3DUnetCNN/examples/sppin/aligned/PT_81/20200129/PT_81_DWI_b100_20200129.nii.gz'],
  'label': './aligned/PT_81/20200129/PT_81_NB_20200129.nii.gz'},
 {'image': ['/lustre/work/aizenberg/dgellis/MICCAI23/code/3DUnetCNN/examples/sppin/aligned/PT_81/20200515/PT_81_T1_gd_20200515.nii',
   '/lustre/work/aizenberg/dgellis/MICCAI23/code/3DUnetCNN/examples/sppin/aligned/PT_81/20200129/PT_81_T2_20200129.nii.gz',
   '/lustre/work/aizenberg/dgellis/MICCAI23/code/3DUnetCNN/examples/sppin/aligned/PT_81/20200515/PT_81_DWI_b0_20200515.nii.gz',
   '/lustre/work/aizenberg/dgellis

In [50]:
import ants

In [77]:
t2 = ants.image_read('/lustre/work/aizenberg/dgellis/MICCAI23/code/3DUnetCNN/examples/sppin/aligned/PT_81/20200515/PT_81_t2_20200515.nii.gz')

In [78]:
t1 = ants.image_read(item['image'][0])

In [79]:
result = ants.registration(fixed=t1, moving=t2, type_of_transform='QuickRigid')

In [80]:
ants.image_write(result['warpedmovout'], "test.nii.gz")

In [81]:
image = loader("test.nii.gz")

In [82]:
image.affine

tensor([[-9.1146e-01, -4.1720e-05,  1.4998e-04,  1.7388e+02],
        [ 4.2286e-05, -9.1144e-01,  9.4451e-03,  1.5315e+02],
        [ 9.0294e-05,  5.7029e-03,  1.5095e+00, -4.9823e+02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],
       dtype=torch.float64)

In [84]:
loader('/lustre/work/aizenberg/dgellis/MICCAI23/code/3DUnetCNN/examples/sppin/aligned/PT_81/20200515/PT_81_t2_20200515.nii.gz').affine

tensor([[-9.1146e-01, -4.1720e-05,  1.4998e-04,  1.7388e+02],
        [ 4.2286e-05, -9.1144e-01,  9.4451e-03,  1.5315e+02],
        [ 9.0294e-05,  5.7029e-03,  1.5095e+00, -4.9823e+02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],
       dtype=torch.float64)

In [83]:
import shutil

In [58]:
shutil.move("test.nii.gz", '/lustre/work/aizenberg/dgellis/MICCAI23/code/3DUnetCNN/examples/sppin/aligned/PT_81/20200515/PT_81_t2_20200515.nii.gz')

'/lustre/work/aizenberg/dgellis/MICCAI23/code/3DUnetCNN/examples/sppin/aligned/PT_81/20200129/PT_81_T2_20200129.nii.gz'

In [6]:
# get the training filenames
config["training_filenames"] = list()
ground_truth_filenames = sorted(glob.glob("./preprocessed/*/*/*.nii.gz"))
for label_filename in ground_truth_filenames:
    subject, visit = label_filename.split("/")[-3:-1]
    filenames = sorted(glob.glob(os.path.join("train", subject, visit, f"*.nii*")))
    n_features = len(filenames) 
    feature_modalities = ["_".join(fn.split("_")[3:-1]) for fn in filenames]
    t1_filename = filenames[feature_modalities.index("T1_gd")]
    assert os.path.exists(t1_filename)
    config["training_filenames"].append({"image": t1_filename, "label": label_filename})

with open(config_filename, "w") as op:
    json.dump(config, op, indent=4)

In [12]:
import torch

In [6]:
torch.quantile(torch.rand(100), 1.1)

RuntimeError: quantile() q must be in the range [0, 1] but got 1.1

In [16]:
torch.pi/4

0.7853981633974483

In [15]:
45*8

360

In [17]:
360/16

22.5

In [19]:
(0.1/torch.pi) * 180

5.729577951308232